<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Topology_Demands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install networkx openpyxl

In [20]:
import networkx as netx
import pandas as pd
import numpy as np
import os
from google.colab import files

from scipy.stats import spearmanr
from networkx import *

def csv_to_graph(path, demands_path):
  Df = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Demands = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  df = Df.iloc[1:].copy()
  demands = Demands.iloc[1:].copy()
  df['source'] = df['source'].astype(int)
  df['target'] = df['target'].astype(int)
  df['length'] = df['length'].astype(float)
  demands['source'] = demands['source'].astype(int)
  demands['target'] = demands['target'].astype(int)
  demands['bitrate'] = demands['bitrate'].astype(int) // 100
  demands_grouped = demands.groupby(['source', 'target']).sum().reset_index()

  sources = df['source'].tolist()
  targets = df['target'].tolist()
  lengths = df['length'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_matrix = np.zeros((num_vertices, num_vertices))
  demand_matrix = np.zeros((num_vertices, num_vertices))

  for source, target, length in zip(sources, targets, lengths):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_matrix[new_source, new_target] = length
    adj_matrix[new_target, new_source] = length

  demands_grouped['source'] = demands_grouped['source'].map(vertex_mapping)
  demands_grouped['target'] = demands_grouped['target'].map(vertex_mapping)
  for _, row in demands_grouped.iterrows():
    demand_matrix[row['source'], row['target']] = row['bitrate']

  nodes_params = []

  degrees = list(np.count_nonzero(adj_matrix, axis=1))
  nodes_params.append(degrees)

  source_demands = list(np.sum(demand_matrix, axis=1))
  nodes_params.append(source_demands)

  target_demands = list(np.sum(demand_matrix, axis=0))
  nodes_params.append(target_demands)

  return (adj_matrix, demand_matrix, nodes_params)

def graph_topology(main_graph):
  topology = []
  if netx.is_connected(main_graph):
    graph = main_graph
  else:
    components = list(netx.connected_components(main_graph))
    largest_component = max(components, key=len)
    graph = main_graph.subgraph(largest_component)

  num_nodes = graph.number_of_nodes()
  topology.append(num_nodes)

  num_edges = graph.number_of_edges()
  topology.append(num_edges)

  degrees = graph.degree()
  avg_degree = np.mean(graph.degree(), axis = 0)[1]
  topology.append(avg_degree)

  diameter = netx.diameter(graph)
  topology.append(diameter)

  avg_shortest_path_length = netx.average_shortest_path_length(graph)
  topology.append(avg_shortest_path_length)

  avg_clustering_coefficient = netx.average_clustering(graph)
  topology.append(avg_clustering_coefficient)

  local_clustering = list(netx.clustering(graph).values())
  std_clustering_coefficient = np.std(local_clustering)
  topology.append(std_clustering_coefficient)

  node_betweenness = list(netx.betweenness_centrality(graph).values())
  min_node_betweenness = min(node_betweenness)
  topology.append(min_node_betweenness)
  max_node_betweenness = max(node_betweenness)
  topology.append(max_node_betweenness)
  avg_node_betweenness = sum(node_betweenness) / num_nodes
  topology.append(avg_node_betweenness)

  edge_betweenness = list(netx.edge_betweenness_centrality(graph).values())
  min_edge_betweenness = min(edge_betweenness)
  topology.append(min_edge_betweenness)
  max_edge_betweenness = max(edge_betweenness)
  topology.append(max_edge_betweenness)
  avg_edge_betweenness = sum(edge_betweenness) / num_edges
  topology.append(avg_edge_betweenness)

  return topology

def statistic_dataframe(base_path):
  Output = []
  csv_tables = []
  for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    csv_path = os.path.join(folder_path, 'links.csv')
    csv_path_demands = os.path.join(folder_path, 'demands.csv')
    csv_tables.append(folder_name)

    adj_matrix = csv_to_graph(csv_path, csv_path_demands)
    graph = netx.from_numpy_array(adj_matrix)

    Output.append(graph_topology(graph))

  labels = ['Number of Nodes', 'Number of Edges', 'Average Node Degree', 'Diameter(hops)', 'Average Shortest Path Length (Hops)', 'Average Clustering Coefficient', 'Standard Deviation of Clustering Coefficient', 'Minimum Node Betweenness Centrality', 'Maximum Node Betweenness Centrality', 'Average Node Betweenness Centrality', 'Minimum Edge Betweenness Centrality', 'Maximum Edge Betweenness Centrality', 'Average Edge Betweenness Centrality']
  Table = pd.DataFrame(Output, index = csv_tables ,columns = labels)

  return Table

def demands_analysis(base_path):
  Output = []
  csv_tables = []
  for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    csv_path = os.path.join(folder_path, 'links.csv')
    csv_path_demands = os.path.join(folder_path, 'demands.csv')
    csv_tables.append(folder_name)

    (adj_matrix, demand_matrix, nodes_params) = csv_to_graph(csv_path, csv_path_demands)
    degrees = nodes_params[0]
    source_demands = nodes_params[1]
    target_demands = nodes_params[2]

    tar_corr, tar_p_value = spearmanr(degrees, target_demands)
    src_corr, src_p_value = spearmanr(degrees, source_demands)

    output = [round(tar_corr, 3), round(tar_p_value, 3), round(src_corr , 3), round(src_p_value,3)]
    Output.append(output)

  labels = ['Target Spearman Corr', 'Target Spearman P-Value', 'Source Spearman Corr', 'Source Spearman P-Value']
  Table = pd.DataFrame(Output, index = csv_tables ,columns = labels)

  return Table


def excel(dataframe, file_name):
  dataframe.to_excel(f'{file_name}.xlsx', index = True)
  files.download(f'{file_name}.xlsx')

path_to_folder = '/content/drive/MyDrive/Кола'
#demands_analysis(path_to_folder)
excel(demands_analysis(path_to_folder), 'Degree_demands_corr')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>